In [ ]:
#Proceso04
#limpieza especificas para departamentos
#visualizar mapa palermo y departamentos
#recupero de muestra para RL

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import rtree
import folium
import joypy

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style = "whitegrid")

from geopandas.tools import sjoin
from shapely.geometry import shape,mapping,Point,Polygon

pd.options.display.max_rows = 10

In [ ]:
#funciones graficas

def plot_dist(lista):
    '''
    Input: serie
    Return: subplots con la distribucion original boxplot e hist&kde
    '''
    plt.figure(figsize=(12,8))
    
    plt.subplot(211)
    sns.boxplot(lista)
    plt.title(f'{lista.name}\n Max={np.max(lista):.0f} Median={np.median(lista):.0f} Min={np.min(lista):.0f}')

    plt.subplot(212)
    sns.distplot(lista)

    plt.tight_layout()
    plt.show()

In [ ]:
#geopandas
def from_wkt(df, wkt_column):
    import shapely.wkt
    df["coordinates"]= df[wkt_column].apply(shapely.wkt.loads)
    gdf = gpd.GeoDataFrame(barrios, geometry='coordinates')
    return gdf

def from_x_y(df, x, y):
    gdf = gpd.GeoDataFrame(df.drop([x, y], axis=1),
                                crs={'init': 'epsg:4326'},
                                geometry=[Point(xy) for xy in zip(df[x], df[y])])
    return gdf

In [ ]:
#geopandas distancia entre polygono y puntos
def distancia(poligono,punto):
    dist = poligono.geometry.distance(punto)
    dist_sort = dist.sort_values(ascending=True)
    
    return dist_sort[0]

In [ ]:
#levatamos los barrios
barrios = pd.read_csv("../data/barrios.csv", encoding='latin1')
barrios = from_wkt(barrios, "WKT")

In [ ]:
barrios.head()

In [ ]:
#recuperamos la salida del proceso03
propiedades = pd.read_csv('../data/out_proceso03.csv', decimal=".")

#se aplican nuevos filtros para seleccionar el barrio en que vamos a trabajar -> palermo
analizar = propiedades[(propiedades['provincia'] == 'capital federal') & 
                       (propiedades['tipo_propiedad'] == 'departamento') &
                       (propiedades['ciudad_barrio'] == 'palermo') &
                       (propiedades['emprendimiento'] == False) &
                       (propiedades['lat'].notnull()) &
                       (propiedades['lon'].notnull()) &
                       (propiedades['valor_m2'] > 1300) & 
                       (propiedades['valor_m2'] < 5300) & 
                       (propiedades['sup_m2'] > 10) &
                       (propiedades['sup_m2'] < 600)
                      ].copy()

#se descartan posibles errores de carga sobre tipo de propiedad
analizar.loc[analizar['titulo'].str.contains("chalet|duplex|ph"), 'tipo_propiedad'] = "error"
analizar.loc[analizar['descripcion'].str.contains("chalet|duplex|ph"), 'tipo_propiedad'] = "error"

#nos quedamos solo con los departamentos
analizar = analizar[(analizar['tipo_propiedad'] == 'departamento')]

dfTemp = pd.DataFrame(analizar['descripcion'].copy())

#dfTemp['titulo'].replace('( )', '', regex=True, inplace=True)
dfTemp['descripcion'].replace('( )', '', regex=True, inplace=True)
dfTemp.columns=['descripciontrip']
analizar = analizar.join(dfTemp)

# Eliminar duplicados en descripcion y titulo
analizar.drop_duplicates(['descripciontrip'], inplace=True)
analizar.drop_duplicates(['titulo'], inplace=True)

analizar = from_x_y(analizar, 'lon', 'lat')

In [ ]:
analizar.head()

In [ ]:
analizar.reset_index(drop=True, inplace=True)
analizar.shape

In [ ]:
#poligono del aeroparque
libertador_alcorta = gpd.GeoDataFrame.from_file("../data/aeroparque/aeroparque_poly.shp")

#departamentos en aeroparque
en_aeroparque = gpd.sjoin(analizar,libertador_alcorta,how="inner")

#eliminamos rows de deptos sobre el aeroparque
analizar.drop(analizar.index[[en_aeroparque.index.values]],inplace=True)

analizar.shape

In [ ]:
#nos quedamos solo con los departamentos que se encuentran dentro de palermo
palermo = barrios[barrios['BARRIO']=='PALERMO']

analizar = analizar[analizar.intersects(palermo.unary_union)]

analizar.shape

In [ ]:
m = folium.Map([-34.606359, -58.443863], zoom_start=12, tiles="OpenStreetMap")
folium.GeoJson(palermo.to_json()).add_to(m)
folium.GeoJson(analizar.to_json()).add_to(m)
m

In [ ]:
sns.heatmap(analizar[['precio','sup_m2','valor_m2','expensas','dormitorios','ambientes','seguridad','banos']].corr(), square=True, annot=True, cmap="YlGnBu")

In [ ]:
analizar.groupby(['ciudad_barrio']).agg({'valor_m2':['count','mean','max','min','std']})

In [ ]:
plot_dist(analizar.valor_m2)

In [ ]:
#sup_m2 vs precio
muestra = analizar.sample(frac=0.50)

# Gráfico con línea de regresión
plt.figure(figsize = (20,10))
sns.regplot(x=muestra["sup_m2"], y=muestra["precio"])

In [ ]:
#sup_m2 vs valor_m2
muestra = analizar.sample(frac=0.50)

# Gráfico con línea de regresión
plt.figure(figsize = (20,10))
sns.regplot(x=muestra["sup_m2"], y=muestra["valor_m2"])

In [ ]:
#valor_m2 -> palermo vs palermo sin barrios vs palermo con barrio

#palermo
p_psb_pcb = analizar[['ciudad_barrio','valor_m2','expensas']].copy()
p_psb_pcb.columns=['barrio','valor_m2','expensas']

#palermo sin barrio
dfTemp = analizar[analizar['barrio'].isnull()][['barrio','valor_m2','expensas']].copy()
dfTemp['barrio'] = 'sin barrio'

p_psb_pcb = pd.concat([p_psb_pcb, dfTemp], sort=True, axis=0)

#palermo con palermo chico
dfTemp = analizar[analizar['barrio']=="palermo chico"][['barrio','valor_m2','expensas']].copy()

p_psb_pcb = pd.concat([p_psb_pcb, dfTemp], sort=True, axis=0)

#palermo con palermo chico
dfTemp = analizar[analizar['barrio']=="palermo soho"][['barrio','valor_m2','expensas']].copy()

p_psb_pcb = pd.concat([p_psb_pcb, dfTemp], sort=True, axis=0)

#palermo con palermo hollywood
dfTemp = analizar[analizar['barrio']=="palermo hollywood"][['barrio','valor_m2','expensas']].copy()

p_psb_pcb = pd.concat([p_psb_pcb, dfTemp], sort=True, axis=0)

fig, axes = joypy.joyplot(p_psb_pcb, by="barrio", column="valor_m2", x_range=[500,8000], figsize=(20,10))

In [ ]:
#palermo con palermo viejo
dfTemp = analizar[analizar['barrio']=="palermo viejo"][['barrio','valor_m2','expensas']].copy()

p_psb_pcb = pd.concat([p_psb_pcb, dfTemp], sort=True, axis=0)


fig, axes = joypy.joyplot(p_psb_pcb, by="barrio", column="valor_m2",
                          hist="True", bins=50, overlap=0,
                          grid=True, legend=False, x_range=[600,6000], figsize=(20,10), ylabelsize=18, xlabelsize=18)

In [ ]:
p_psb_pcb.groupby('barrio').count()

In [ ]:
p_psb_pcb.groupby('barrio')['valor_m2'].agg(['mean','max','min']).plot.bar()

In [ ]:
p_psb_pcb.groupby('barrio')['expensas'].agg(['mean','max','min']).plot.bar()

In [ ]:
# Ploteamos el histograma de Precio
plt.figure(figsize=(10,6))

plt.hist(analizar['precio'], bins=35, color='lightblue')
plt.xlabel('Precio aprox USD')
plt.ylabel('Frecuencia')
plt.title('Histograma - Precio Medio: u$d ' + str("{0:.2f}".format(analizar['precio'].mean())))
plt.grid(True)
plt.show()

In [ ]:
# Ploteamos el histograma de valor_m2
plt.figure(figsize=(10,6))

plt.hist(analizar['valor_m2'], bins=35, color='lightblue')
plt.xlabel('Precio aprox USD')
plt.ylabel('Frecuencia')
plt.title('Histograma - Valor M2 Medio: u$d ' + str("{0:.2f}".format(analizar['valor_m2'].mean())))
plt.grid(True)
plt.show()

In [ ]:
# Ploteamos el histograma de expensas
plt.figure(figsize=(10,6))

#se reemplaza string np.nan por 0
analizar['expensas'].replace(to_replace=np.nan, value=0, inplace=True)

plt.hist(analizar['expensas'], bins=10, color='lightblue')
plt.xlabel('Expensas')
plt.ylabel('Frecuencia')
plt.title('Histograma - Media Expensas: $' + str("{0:.2f}".format(analizar['expensas'].mean())))
plt.grid(True)
plt.show()

In [ ]:
# Ploteamos el histograma con Sup Cubierta

plt.figure(figsize=(10,6))

#se reemplaza string np.nan por 0
analizar['sup_m2_cubiertos'].replace(to_replace=np.nan, value=0, inplace=True)

plt.hist(analizar['sup_m2_cubiertos'], bins=35, color='lightblue')
plt.xlabel('Sup cubierta')
plt.ylabel('Frecuencia')
plt.title('Histograma - Sup Media: ' + str("{0:.2f}".format(analizar['sup_m2_cubiertos'].mean())) + 'm2')
plt.grid(True)
plt.show()

In [ ]:
#levantamos los poligonos generados para generar nuevas columnas
libertador_alcorta = gpd.GeoDataFrame.from_file("../data/avenidas/avLibertador_avAlcorta_poly.shp")

plaza_parque = gpd.GeoDataFrame.from_file("../data/plaza/plazayparques_poly.shp")

subte = gpd.GeoDataFrame.from_file("../data/subte/subte_poly.shp")

metrobus = gpd.GeoDataFrame.from_file("../data/metrobus/metrobus_poly.shp")

In [ ]:
#segun distancia al plazas y espacios verdes

dfTemp = pd.DataFrame(analizar.apply(lambda x: distancia(plaza_parque, x['geometry']), axis=1))

#se renombran la columna nueva
dfTemp.columns=['dist_plazas']

#se suma al df la nueva columna dist_plazas
analizar = analizar.join(dfTemp)

m = folium.Map([-34.606359, -58.443863], zoom_start=12, tiles="OpenStreetMap")
folium.GeoJson(plaza_parque.to_json()).add_to(m)
m


In [ ]:
#segun distancia al subte

dfTemp = pd.DataFrame(analizar.apply(lambda x: distancia(subte, x['geometry']), axis=1))

#se renombran la columna nueva
dfTemp.columns=['dist_subte']

#se suma al df la nueva columna dist_subte
analizar = analizar.join(dfTemp)

m = folium.Map([-34.606359, -58.443863], zoom_start=12, tiles="OpenStreetMap")
folium.GeoJson(subte.to_json()).add_to(m)
m


In [ ]:
#segun distancia al metrobus

dfTemp = pd.DataFrame(analizar.apply(lambda x: distancia(metrobus, x['geometry']), axis=1))

#se renombran la columna nueva
dfTemp.columns=['dist_metrobus']

#se suma al df la nueva columna dist_metrobus
analizar = analizar.join(dfTemp)

m = folium.Map([-34.606359, -58.443863], zoom_start=12, tiles="OpenStreetMap")
folium.GeoJson(metrobus.to_json()).add_to(m)
m


In [ ]:
# dentro de las 3 cuadras de av libertador y av alcorta
dfTemp = pd.DataFrame(analizar.apply(lambda x: distancia(libertador_alcorta, x['geometry']), axis=1))

#se renombran la columna nueva
dfTemp.columns=['dist_lib_alc']

#se suma al df la nueva columna dist_lib_alc
analizar = analizar.join(dfTemp)


m = folium.Map([-34.606359, -58.443863], zoom_start=12, tiles="OpenStreetMap")
folium.GeoJson(libertador_alcorta.to_json()).add_to(m)
m


In [ ]:
analizar

In [ ]:
#se guarda csv completo
analizar.to_dense().to_csv("../data/out_proceso04.csv", index = False, sep=',', encoding='utf-8')